# [Advent of Code 2020: Day 11](https://adventofcode.com/2020/day/11)

## --- Day 11: Seating System ---

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (`.`), an empty seat (`L`), or an occupied seat (`#`). For example, the initial seat layout might look like this:

```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL

```

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the **number of occupied seats** adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

*   If a seat is **empty** (`L`) and there are **no** occupied seats adjacent to it, the seat becomes **occupied**.
*   If a seat is **occupied* (`#`) and **four or more** seats adjacent to it are also occupied, the seat becomes **empty**.
*   Otherwise, the seat's state does not change.

Floor (`.`) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied.

After a second round, the seats with four or more occupied adjacent seats become empty again.

This process continues for three more rounds:

```
0           1           2           3           4           5
L.LL.LL.LL  #.##.##.##  #.LL.L#.##  #.##.L#.##  #.#L.L#.##  #.#L.L#.##
LLLLLLL.LL  #######.##  #LLLLLL.L#  #L###LL.L#  #LLL#LL.L#  #LLL#LL.L#
L.L.L..L..  #.#.#..#..  L.L.L..L..  L.#.#..#..  L.L.L..#..  L.#.L..#..
LLLL.LL.LL  ####.##.##  #LLL.LL.L#  #L##.##.L#  #LLL.##.L#  #L##.##.L#
L.LL.LL.LL  #.##.##.##  #.LL.LL.LL  #.##.LL.LL  #.LL.LL.LL  #.#L.LL.LL
L.LLLLL.LL  #.#####.##  #.LLLL#.##  #.###L#.##  #.LL#L#.##  #.#L#L#.##
..L.L.....  ..#.#.....  ..L.L.....  ..#.#.....  ..L.L.....  ..L.L.....
LLLLLLLLLL  ##########  #LLLLLLLL#  #L######L#  #L#LLLL#L#  #L#L##L#L#
L.LLLLLL.L  #.######.#  #.LLLLLL.L  #.LL###L.L  #.LLLLLL.L  #.LLLLLL.L
L.LLLLL.LL  #.#####.##  #.#LLLL.##  #.#L###.##  #.#L#L#.##  #.#L#L#.##

```

At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count **`37`** occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. **How many seats end up occupied?**

In [1]:
import unittest
from functools import lru_cache
from IPython.display import Markdown, display

from aoc_puzzle import AocPuzzle

class SeatingSystem(AocPuzzle):
    
    MAX_OCCUPIED = 4
    
    def parse_data(self, raw_data):
        self.width = raw_data.index('\n')
        self.height = raw_data.count('\n') + 1
        
        self.data = raw_data.replace('\n','')

        self.FLOOR = '.'
        self.IN_USE = '#'
        self.VACANT = 'L'
    
    @lru_cache(maxsize = 1000)
    def is_valid(self, coord):
        x,y = coord
        return x >= 0 and x < self.width and y >= 0 and y < self.height 
    
    @lru_cache(maxsize = 1000)
    def get_coords(self, index):
        x = index % (self.width)
        y = int(index / (self.width))
        
        return (x,y)
    
    def get_seat(self, coord):
        x,y = coord        
        index = x + (y * (self.width))
        return self.data[index]
    
    @lru_cache(maxsize = 1000)    
    def get_adjacent_seat_coords(self, coord):
        x, y = coord
        adjacent_seat_coords = [(x,y+1),   # up
                                (x+1,y+1),   # up-right
                                (x+1,y),   # right
                                (x+1,y-1),  # down-right
                                (x,y-1),  # down
                                (x-1,y-1), # down-left
                                (x-1,y),  # left
                                (x-1,y+1)]  # left-up
        
        valid_adjacent_seat_coords = []
        
        for seat_coord in adjacent_seat_coords:
            if self.is_valid(seat_coord):
                valid_adjacent_seat_coords.append(seat_coord)
        
        return valid_adjacent_seat_coords
    
    def get_adjacent_seats(self, index): 
        seat_coord = self.get_coords(index)
        adjacent_seat_coords = self.get_adjacent_seat_coords(seat_coord)        
        adjacent_seats = []
        for coord in adjacent_seat_coords:            
            adjacent_seats.append(self.get_seat(coord))            
        return adjacent_seats
        
    def apply_rules(self):
        new_data = list(self.data)
        for index, seat in enumerate(self.data):
            if (seat != self.FLOOR):
                adjacent_seats = self.get_adjacent_seats(index)

                # If a seat is occupied and MAX_OCCUPIED or more seats adjacent to it are
                # also occupied, the seat becomes empty
                if (seat == self.IN_USE):
                    adjacent_occupied = adjacent_seats.count(self.IN_USE)
                    if adjacent_occupied >= self.MAX_OCCUPIED:
                        new_data[index] = self.VACANT

                # If a seat is empty and there are no occupied seats adjacent to it
                # the seat becomes occupied
                elif (seat == self.VACANT):
                    if self.IN_USE not in adjacent_seats:
                        new_data[index] = self.IN_USE
                    
        self.data = ''.join(new_data)
    
    def run(self, max_tries=100, output=False, debug=False):
        tries = 0 
        prev_data = ''
        while (tries < max_tries) and (prev_data != self.data):
            if debug:
                print(tries)
                for i in range(self.height+1):
                    start = self.width * i
                    end = start + self.width
                    print(self.data[start:end])
            prev_data = self.data
            self.apply_rules()
            tries += 1
            
        seats_in_use = self.data.count(self.IN_USE)
        
        if output:
            display(Markdown(f'### Result is `{seats_in_use}, tries: {tries}`'))
        return seats_in_use
        
class TestBasic(unittest.TestCase):
    
    in_data = 'L.LL.LL.LL\nLLLLLLL.LL\nL.L.L..L..\nLLLL.LL.LL\nL.LL.LL.LL\nL.LLLLL.LL\n..L.L.....\nLLLLLLLLLL\nL.LLLLLL.L\nL.LLLLL.LL'

    def test_parse_data(self):
        exp_out = 'L.LL.LL.LLLLLLLLL.LLL.L.L..L..LLLL.LL.LLL.LL.LL.LLL.LLLLL.LL..L.L.....LLLLLLLLLLL.LLLLLL.LL.LLLLL.LL'
        ss = SeatingSystem(self.in_data)
        self.assertEqual(ss.data, exp_out)
        self.assertEqual(ss.width, 10)
        self.assertEqual(ss.height, 10)
        
    def test_simulation(self):
        exp_data = '#.#L.L#.###LLL#LL.L#L.#.L..#..#L##.##.L##.#L.LL.LL#.#L#L#.##..L.L.....#L#L##L#L##.LLLLLL.L#.#L#L#.##'
        exp_out = 37
        ss = SeatingSystem(self.in_data)
        self.assertEqual(ss.run(), exp_out)
        self.assertEqual(ss.data, exp_data)
        
unittest.main(argv=[""], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


In [2]:
ss = SeatingSystem("input/d11.txt")
ss.run(output=True)

### Result is `2476, tries: 84`

2476

## --- Part Two ---

As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about **the first seat they can see** in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the **first seat** in each of those eight directions. For example, the empty seat below would see **eight** occupied seats:

```
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....

```

The leftmost empty seat below would only see **one** empty seat, but cannot see any of the occupied ones:

```
.............
.L.L.#.#.#.#.
.............

```

The empty seat below would see **no** occupied seats:

```
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.

```

Also, people seem to be more tolerant than you expected: it now takes **five or more** visible occupied seats for an occupied seat to become empty (rather than **four or more** from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:

```
0           1           2           3           4           5           6
L.LL.LL.LL  #.##.##.##  #.LL.LL.L#  #.L#.##.L#  #.L#.L#.L#  #.L#.L#.L#  #.L#.L#.L#
LLLLLLL.LL  #######.##  #LLLLLL.LL  #L#####.LL  #LLLLLL.LL  #LLLLLL.LL  #LLLLLL.LL
L.L.L..L..  #.#.#..#..  L.L.L..L..  L.#.#..#..  L.L.L..#..  L.L.L..#..  L.L.L..#..
LLLL.LL.LL  ####.##.##  LLLL.LL.LL  ##L#.##.##  ##LL.LL.L#  ##L#.#L.L#  ##L#.#L.L#
L.LL.LL.LL  #.##.##.##  L.LL.LL.LL  #.##.#L.##  L.LL.LL.L#  L.L#.#L.L#  L.L#.LL.L#
L.LLLLL.LL  #.#####.##  L.LLLLL.LL  #.#####.#L  #.LLLLL.LL  #.L####.LL  #.LLLL#.LL
..L.L.....  ..#.#.....  ..L.L.....  ..#.#.....  ..L.L.....  ..#.#.....  ..#.L.....
LLLLLLLLLL  ##########  LLLLLLLLL#  LLL####LL#  LLLLLLLLL#  LLL###LLL#  LLL###LLL#
L.LLLLLL.L  #.######.#  #.LLLLLL.L  #.L#####.L  #.LLLLL#.L  #.LLLLL#.L  #.LLLLL#.L
L.LLLLL.LL  #.#####.##  #.LLLLL.L#  #.L####.L#  #.L#LL#.L#  #.L#LL#.L#  #.L#LL#.L#

```

Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count **`26`** occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, **how many seats end up occupied?**

In [3]:
from fractions import Fraction

class SeatingSystem2(SeatingSystem):
    
    MAX_OCCUPIED = 5
    
    @lru_cache(maxsize = 1000)
    def get_slope(self, coord1, coord2):
        x1, y1 = coord1
        x2, y2 = coord2
        return((x1 - x2, y1 - y2))
    
    def incriment_coord(self, coord, step):
        x,y = coord
        xstep, ystep = step
        return (x+xstep, y+ystep)
    
    @lru_cache(maxsize = 1000)
    def get_step(self, coord):
        x, y = coord
        if x is 0:
            xcd = 0
            ysign = y / abs(y)
            ycd = ysign
            return (xcd, int(ycd))
        elif y is 0:
            xsign = x / abs(x)
            xcd = xsign
            ycd = 0
            return (int(xcd), ycd)
        else:
            xsign = x / abs(x)
            ysign = y / abs(y)
            cf = Fraction(x, y)
            xcd = abs(cf.numerator) * xsign
            ycd = cf.denominator * ysign
            
            return (int(xcd), int(ycd))
    
    @lru_cache(maxsize = 1000)
    def get_step_to(self, coord1, coord2):
        """ Returns a step for coordinate 2 that is in the direction of coordinate 1 """
        if coord1 is coord2:
            return (0,0)
        
        x1, y1 = coord1
        x2, y2 = coord2        
        dist = self.get_slope(coord1, coord2)
        xstep, ystep = self.get_step(dist)
        
        return (xstep, ystep)
    
    def get_next_visible_seat(self, seat_coord, coord):        
        step = self.get_step_to(coord, seat_coord)        
        next_coord = self.incriment_coord(seat_coord, step)
        
        while self.is_valid(next_coord):
            seat = self.get_seat(next_coord)
            if seat != self.FLOOR:
                return seat
            next_coord = self.incriment_coord(next_coord, step)
        return self.FLOOR        

    def get_adjacent_seats(self, index):        
        seat_coord = self.get_coords(index)
        adjacent_seat_coords = self.get_adjacent_seat_coords(seat_coord)        
        adjacent_seats = []
        for coord in adjacent_seat_coords:            
            seat = self.get_seat(coord)
            if seat == self.FLOOR:
                seat = self.get_next_visible_seat(seat_coord, coord)
            adjacent_seats.append(seat)
            
        return adjacent_seats        

class TestBasic(unittest.TestCase):
        
    def test_simulation(self):
        in_data = 'L.LL.LL.LL\nLLLLLLL.LL\nL.L.L..L..\nLLLL.LL.LL\nL.LL.LL.LL\nL.LLLLL.LL\n..L.L.....\nLLLLLLLLLL\nL.LLLLLL.L\nL.LLLLL.LL'
        exp_data= '#.L#.L#.L##LLLLLL.LLL.L.L..#..##L#.#L.L#L.L#.LL.L##.LLLL#.LL..#.L.....LLL###LLL##.LLLLL#.L#.L#LL#.L#'
        exp_out = 26
        ss = SeatingSystem2(in_data)
        self.assertEqual(ss.run(), exp_out)
        self.assertEqual(ss.data, exp_data)
        
unittest.main(argv=[""], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


In [4]:
ss = SeatingSystem2("input/d11.txt")
ss.run(output=True)

### Result is `2257, tries: 90`

2257